# TP3 : Modélisation - Correction

In [2]:
# Librairies à importer pour utiliser JuMP avec le solver GLPK
using JuMP
using GLPK

# Définition de constantes pour le statut de résolution du problème
const OPTIMAL = JuMP.MathOptInterface.OPTIMAL
const INFEASIBLE = JuMP.MathOptInterface.INFEASIBLE
const UNBOUNDED = JuMP.MathOptInterface.DUAL_INFEASIBLE;

## Exercice 1 : Équipe de superhéros

Pour combattre les aliens qui envahissent la terre, il faut créer une équipe de superhéros travaillant main dans la main. Malheureusement, certains superhéros sont ennemis et ne peuvent donc pas faire équipe...  Combien de superhéros peuvent aller combattre les aliens ? Il faut trouver l'équipe la plus importante possible sans ennemis... Le sort de la terre en dépend !

Voici la liste des superhéros : 
   * Batman
   * Superman
   * Catwoman
   * Flash
   * Wonder woman
   * Black Panther
   * Captain America
   * Daredevil
   * Elektra
   * Hulk

Et voici maintenant la liste des ennemis jurés : 
   * Batman et Flash
   * Catwoman et Captain America
   * Daredevil et Elektra
   * Hulk et Batman
   * Catwoman et Wonder woman
   * Black Panther et Hulk
   * Superman et Flash
   * Superman et Elektra
   * Flash et Daredevil
   * Wonder woman et Captain America
   * Daredevil et Hulk
   * Batman et Captain America
   * Batman et Wonder woman
   * Black Panther et Wonder woman



**Remarque :** Il est possible de définir des variables indexées par un tableau de chaînes de caractères. Par exemple, il est possible de déclarer les variables `@variable(m,x[Superhéros])`





In [3]:
#=
    DONNÉES
=#

Superhéros = [
 "Batman",
 "Superman",
 "Catwoman",
 "Flash",
 "Wonder woman",
 "Black Panther",
 "Captain America",
 "Daredevil",
 "Elektra",
 "Hulk"
]


Ennemis = [
  ["Batman", "Flash"],
  ["Catwoman", "Captain America"],
  ["Daredevil", "Elektra"],
  ["Hulk", "Batman"],
  ["Catwoman", "Wonder woman"],
  ["Black Panther", "Hulk"],
  ["Superman", "Flash"],
  ["Superman", "Elektra"],
  ["Flash", "Daredevil"],
  ["Wonder woman", "Captain America"],
  ["Daredevil", "Hulk"],
  ["Batman", "Captain America"],
  ["Batman", "Wonder woman"],
  ["Black Panther", "Wonder woman"]
];


In [4]:
##################
#   Correction   #
##################


function foundBestTeam(Superhéros, Ennemis)

    superteam = Model(GLPK.Optimizer)

    # Une varible binaire par super héros
    @variable(superteam,x[Superhéros], Bin)

    #La fonction objectif : On cherche à minimiser le nombre de rouleaux decoupes.
    @objective(superteam, Max, sum(x[s] for s in Superhéros))

    # une contrainte par paire d'ennemis
    for en in Ennemis
        @constraint(superteam, x[en[1]] + x[en[2]] <= 1)
    end
        
    print(superteam)
    JuMP.optimize!(superteam)
    
    println("Nombre de superhéros pour combattre les aliens : ", round(Int,objective_value(superteam)))
        
    #Affichage du résultat
    println("Voici la liste des superhéros :")
    for h in Superhéros
        if JuMP.value(x[h]) == 1
            println(h)
        end
    end
end

foundBestTeam(Superhéros, Ennemis)

Nombre de superhéros pour combattre les aliens : 5
Voici la liste des superhéros :
Batman
Superman
Catwoman
Black Panther
Daredevil


## Exercice 2 : Pause sudoku


Le but de cet exercice est de résoudre la grille de sudoku ci-dessous **à l'aide de la programmation linéaire en nombres entiers !**


**Remarque :** On utilisera comme variable `x[i,j,k]` qui vaut 1 si la case (i,j) contient la valeur k, et 0 sinon.

In [16]:
#=
    DONNÉES
=#

Grille = [
  5 3 0 0 7 0 0 0 0;
  6 0 0 1 9 5 0 0 0;
  0 9 8 0 0 0 0 6 0;
  8 0 0 0 6 0 0 0 3;
  4 0 0 8 0 3 0 0 1;
  7 0 0 0 2 0 0 0 6;
  0 6 0 0 0 0 2 8 0;
  0 0 0 4 1 9 0 0 5;
  0 0 0 0 8 0 0 7 9
];

In [14]:
##################
#   Correction   #
##################


function solve_sudoku(grille)
    
    sudoku = Model(GLPK.Optimizer)

    # x[i,j,k] vaut 1 si la valeur k est dans la case (i,j)
    @variable(sudoku,x[1:9,1:9,1:9], Bin)

    #La fonction objectif : il n'y en pas besoin car on cherche simplement une solution réalisable
    
    for i in 1:9, j in 1:9
        @constraint(sudoku, sum(x[i,j,k] for k in 1:9) == 1)
    end
    
    # Pour chaque valeur
    for k in 1:9
        
        # Pour chaque colonne
        for i in 1:9
            @constraint(sudoku, sum(x[i,j,k] for j in 1:9) == 1)
        end
        
        # Pour chaque ligne
        for j in 1:9
            @constraint(sudoku, sum(x[i,j,k] for i in 1:9) == 1)
        end
        
        # Pour chaque carré 
        for lc in 0:2, cc in 0:2
            @constraint(sudoku, sum(x[3 * lc + i, 3 * cc + j,k] for i in 1:3, j in 1:3 ) == 1)
        end
    end
    
    # Valeurs initiales
    for i in 1:9, j in 1:9
        if grille[i,j] != 0
            @constraint(sudoku, x[i,j,grille[i,j]] == 1)
        end
    end
    
    #println(sudoku)

    
    JuMP.optimize!(sudoku)
    status = termination_status(sudoku)
    

    if status == INFEASIBLE
        println("Le problème n'est pas réalisable")
    else
        println("Grille résolue !")
        sol = zeros(Int, 9, 9)
        for i in 1:9, j in 1:9, k in 1:9
            if JuMP.value(x[i,j,k]) ≈ 1.
                sol[i, j] = k
            end
        end
        
        #= Affichage de la solution
        for i in 1:9
            for j in 1:9
                print(sol[i, j], " ")
            end
            println("")
        end
        =#
        
        #Affichage évolué       
        A = replace(i -> i == 0 ? "." : string(i), Matrix{Any}(sol))
        nn = size(A)[1]
        n = Int(sqrt(nn))

        digitsLength = length(string(nn))
        A = lpad.(A, digitsLength, " ") # for working properly with n > 4


        header = "+" * prod(["-"^(digitsLength * n + n + 1) * "+" for _ ∈ 1:n])
        println(header)

        # print every line 
        for i ∈ 1:nn
            to_print = ""
            for j ∈ 1:nn
                if j == 1
                    to_print *= "| " * A[i, j] * " "
                else
                    to_print *= A[i, j] * " "
                end
                if j % n == 0
                    to_print *= "| "
                end
            end
            println(to_print)
            if i % n == 0
                println(header)
            end
        end
        
    end
end
        
solve_sudoku(Grille)

Grille résolue !
+-------+-------+-------+
| 5 3 4 | 6 7 8 | 9 1 2 | 
| 6 7 2 | 1 9 5 | 3 4 8 | 
| 1 9 8 | 3 4 2 | 5 6 7 | 
+-------+-------+-------+
| 8 5 9 | 7 6 1 | 4 2 3 | 
| 4 2 6 | 8 5 3 | 7 9 1 | 
| 7 1 3 | 9 2 4 | 8 5 6 | 
+-------+-------+-------+
| 9 6 1 | 5 3 7 | 2 8 4 | 
| 2 8 7 | 4 1 9 | 6 3 5 | 
| 3 4 5 | 2 8 6 | 1 7 9 | 
+-------+-------+-------+
